## Celestial Coordinates

- The celestial sphere
- RA, DEC
- ICRS
- Compute RA, DEC of the sources in the image.
- Use `astropy.coordinates` to transform RA, DEC to Galactic coordinates
- Overlay galactic grid over the wcsaxes image
- Clean up notebook and get it ready for presentation

#### The Celestial System


A useful construct for describing locations of objects in the sky is the celestial sphere, which is considered to have an infinite radius. The center of the earth is the center of the celestial sphere, and the sphere's pole and equatorial plane are coincident with those of the earth. We can specify precise location of objects on the celestial sphere by giving the celestial equivalent of their latitudes and longitudes.

The point on the celestial sphere directly overhead for an observer is the zenith. An imaginary arc passing through the celestial poles and through the zenith is called the observer's meridian. The nadir is the direction opposite the zenith: for example, straight down from a spacecraft to the center of the planet.

Another important feature intersecting the celestial sphere is the **ecliptic** plane. This is the plane in which the Earth orbits the sun, 23.4° from the celestial equator. The great circle marking the intersection of the ecliptic plane on the celestial sphere is where the sun and planets appear to travel, and it's where the Sun and Moon converge during their eclipses (hence the name).


In [ ]:
from IPython.display import Image
Image('celestial_sphere.jpg')


#### Declination and Right Ascension

**Declination** (`DEC`) is the celestial sphere's equivalent of latitude and it is expressed in degrees, as is latitude. For DEC, + and - refer to north and south, respectively. The celestial equator is 0° DEC, and the poles are +90° and -90°.

**Right ascension** `RA` is the celestial equivalent of longitude. RA can be expressed in degrees, but it is more common to specify it in hours, minutes, and seconds of time: the sky appears to turn 360° in 24 hours, or 15° in one hour. So an hour of RA equals 15° of sky rotation.

The zero point for `RA` is one of the points where the ecliptic circle intersects the celestial equator circle. It's defined to be the point where the sun crosses into the northern hemisphere beginning spring: the **vernal equinox**, also known as the first point of Aries, often identified by the symbol of the ram.

The RA and DEC of an object specify its position uniquely on the celestial sphere just as the latitude and longitude do for an object on the Earth's surface. For example, the very bright star Sirius has celestial coordinates 6 hr 45 min RA and -16° 43' DEC.

In [ ]:
Image('celestial_coords.jpg')

#### The International Celestial Reference System

The International Celestial Reference System (ICRS) is the fundamental celestial reference system that has been adopted by the International Astronomical Union (IAU) for high-precision positional astronomy. The ICRS, with its origin at the solar system barycenter and "space fixed" axis directions, is meant to represent the most appropriate coordinate system for positions and motions of celestial objects. RA and DEC measurements can be transformed to the ICRS sytem.

In [ ]:
from astropy import coordinates as coord
import astropy.units as u
import matplotlib.pyplot as plt
%matplotlib inline
#%matplotlib notebook

In [ ]:
from astropy.io import fits
from astropy import table
\

#### Transform x, y to celestial coordinates

Let's go back to our image and the sources we identified in it yesterday.
In this section we will read the X, Y coordinates from the file `sources.csv` and will compute the corresponding celestial coordinates.

`astropy.Table.read` can read files in several different formats, including `csv`.

In [ ]:
sources = table.Table.read('sources.csv')

The variable `sources` is of type `astropy.table.Table`. We can access items in the table using row indices or column names. Let's look at the first 5 rows of the table:

In [ ]:
sources[0:5]

Now let's see what the names of the columns are and read the columns storing the `x` ("xcentroid") and `y` ("ycentroid) coordinates of the objects.

In [ ]:
print(sources.colnames)

In [ ]:
x, y = sources['xcentroid'], sources['ycentroid']

Let's play with the `print` function to display the 2 table columns in different ways.

In [ ]:
#print("x, y columns: ", x, y)
# print("x, y columns:")
# print(x, y)

#for i in range(len(x)):
#    print(x[i], y[i])
    

##for i in range(len(x)):
##    print(x[i], "\t", y[i])

Now that we have the locations of the stars in pixels on the image we will look at how to compute their coordinates on the celestial sphere. Every HST image comes with information about the celestial coordinates of the point the telescope was "looking at" (or pointing) at the time the observation was taken. This informaton is saved in keywords in the FITS file header of the `SCI` extension. What the specific keywords are is irrelevant at the moment. What's important it that the keywords are standardized and there are libraries which can read, interpret them and perform the transformation. One such library is in the `astropy` package and is called `wcs`. The acronym `wcs` stands for "World Coordinate System". In order to use it we need to import it.

In [ ]:
from astropy import wcs

Next we will open the FITS file we worked with yesterday and pass the header of the `SCI` extension (HDU 1) to the WCS function. 

In [ ]:
f = fits.open('ib2j02020_drz.fits')
w = wcs.WCS(f[1].header)

Now we are ready to transform the x, y, coordinates from the image Cartesian coordinate system to the spherical celestial coordinate system.

In [ ]:
sky_coord = w.pixel_to_world(x, y)

In [ ]:
print(type(sky_coord))
sky_coord

In [ ]:
sky_coord.frame

In [ ]:
ra = sky_coord.data.lon
print('ra', ra)

In [ ]:
dec = sky_coord.spherical.lat
print('dec', dec)

#### Matplotlib

`Matplotlib` is a plotting library for the `Python` programming language and its numerical mathematics extension `NumPy`. 

https://matplotlib.org/

We've used the `imshow` command yesterday to display an image. Today we will use the `scatter` command to create a scatter plot of the coordinates. A "scatter" plot is a type of plot or mathematical diagram using Cartesian coordinates to display values for typically two variables for a set of data. 

In [ ]:
%matplotlib inline
%matplotlib notebook
ax = plt.subplot(projection=w)
ax.imshow(f[1].data, vmin=.1, vmax=.5, cmap='Greys', origin='lower', aspect='auto')
ax.scatter(ra, dec, transform=ax.get_transform('world'))
ax.coords[0].set_format_unit(u.degree)

In [ ]:
ax.grid(color='green', ls='solid')

The `astropy.coordinates` package allows transforming betweek standard coordinate systems. For example, we can transform to `Galactic` cooridnates.

In [ ]:
sky_coord.galactic


If we overlay the grid of ther Galactic coordinate system the difference between the two systems is obvious.

In [ ]:
overlay=ax.get_coords_overlay('galactic') 
overlay.grid(color='red', linestyle='solid', alpha=0.5)